In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.util.matrix.*
GlobalConstants.setVerbose(VerboseLevel.STD);

In [2]:
// Block 1: Create network and nodes
val model = Network("MRP")
val delay = Delay(model, "WorkingState")
val queue = Queue(model, "RepairQueue", SchedStrategy.FCFS)
queue.numberOfServers = 2  // 2 repair servers

In [3]:
// Block 2: Create closed class with 3 machines
val closedClass = ClosedClass(model, "Machines", 3, delay)

// Set service rates
delay.setService(closedClass, Exp(0.5))  // Mean working time = 2.0
queue.setService(closedClass, Exp(4.0))  // Mean repair time = 0.25

In [4]:
// Block 3: Define topology (serial routing between working and repair)
model.link(Network.serialRouting(delay, queue))

In [5]:
// Block 4: Solve using CTMC solver
val solver = CTMC(model)
val ctmcAvgTable = solver.avgTable
ctmcAvgTable.print()

CTMC analysis [method: default, lang: java, env: 17.0.14] completed in 0.022659s.
Station       JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
------------------------------------------------------------------------------------------------
WorkingState  Machines  2.66484     2.66484     2.00000     2.00000     1.33242     1.33242     
RepairQueue   Machines  0.33516     0.16655     0.25154     0.25154     1.33242     1.33242     
------------------------------------------------------------------------------------------------


In [6]:
// Display state space
val stateSpace = solver.stateSpace.stateSpace
println("State space:")
stateSpace.print()

State space:
[0  1  2 
 1  0  2 
 2  0  1 
 3  0  0]



In [7]:
// Display infinitesimal generator
val infGen = solver.generator.infGen
println("Infinitesimal generator:")
infGen.print()

Infinitesimal generator:
[-8.0   8.0     0     0 
  0.5  -8.5   8.0     0 
    0   1.0  -5.0   4.0 
    0     0   1.5  -1.5]



In [8]:
// Print formatted infinitesimal generator with state labels
CTMC.printInfGen(infGen, stateSpace)

[ 0.0   1.0   2.0 ] -> [ 1.0   0.0   2.0 ] : 8.0
[ 1.0   0.0   2.0 ] -> [ 0.0   1.0   2.0 ] : 0.5
[ 1.0   0.0   2.0 ] -> [ 2.0   0.0   1.0 ] : 8.0
[ 2.0   0.0   1.0 ] -> [ 1.0   0.0   2.0 ] : 1.0
[ 2.0   0.0   1.0 ] -> [ 3.0   0.0   0.0 ] : 4.0
[ 3.0   0.0   0.0 ] -> [ 2.0   0.0   1.0 ] : 1.5
